# Instructions

This notebook contains code that I used to reamp the GOAT dataset. The basic sound of an electric guitar can be modeled using an amplifier (to model the distortion) and a impulse response (IR) of the speaker cabinet. I personally used the Neural Amp Modeler (NAM) for the amplifier rendering, and the cabinet sim of the NeuralDSP Achetype Nolly plugin for the cabinet IRs. The parameters for each are randomised, allowing for a large amount of variance in the sounds. This code can be modified as well to use any plugins you want!

If you would like to use the NAM, you will need our NAM profile dataset as well (available from Zenodo upon acceptance of our corresponding paper). This dataset contains roughly 7000 publically available NAM profiles covering a large range of different amplifiers and tones, from clean to super distorted. 

We used a CSV file with the alignment results to select for only data which has a f_measure_fine value of .75 or higher to ensure that we only used data with high alignment accuracy. This CSV will be made available soon, but in the meantime you can comment out the relevant lines (marked with ###).

In [1]:
from pedalboard import Pedalboard, Reverb, load_plugin
from pedalboard.io import AudioFile
import os
import json
import hashlib
import librosa
import soundfile as sf
from tqdm import tqdm
import random
import librosa
import pandas as pd
from IPython.display import Audio

SAMPLE_RATE = 44100
BUFFER_SIZE = 128

# REPLACE THESE PATHS WITH PATHS TO YOUR OWN PLUGINS AND NAM DATASET
nam_path = '/Library/Audio/Plug-Ins/VST3/NeuralAmpModeler.vst3'
nolly_path = '/Library/Audio/Plug-Ins/Components/Archetype Nolly X.component'
presets_path = '/Documents/PhD/Datasets/NAMProfileDataset/vstpresets'

nam_presets = os.listdir(presets_path)
remove_name = ".DS_Store.vstpreset"
if remove_name in nam_presets:
    nam_presets.remove(remove_name)

In [11]:
mic_types = ['Dynamic 57', 'Dynamic 421', 'Condenser 184', 'Condenser 414', 'Ribbon 121', 'Ribbon 160', 'Dynamic 57', 'Dynamic 421', 'Condenser 184', 'Condenser 414', 'Ribbon 121', 'Ribbon 160']
room_prob = 0.25
def get_nolly():
    nolly = load_plugin(nolly_path)
    nolly.active_amp_section = False
    #nolly.active_pre_fx_section = False
    nolly.active_eq_section = False
    #nolly.active_post_fx_section = False
    nolly.rhythm_amp_cab_mic_l_type = mic_types[random.randint(0, len(mic_types)-1)]
    nolly.cab_l_position = random.uniform(0, 1)
    nolly.cab_l_distance = random.uniform(0, 1)
    nolly.cab_l_phase = bool(random.randint(0, 1))
    if random.uniform(0, 1) < room_prob:
        nolly.room_l_active = True
        nolly.room_l_send = random.randint(-10, 0)
    else:
        nolly.room_l_active = False
    return nolly

def get_nam():
    nam = load_plugin(nam_path)
    preset_path = os.path.join(presets_path, nam_presets[random.randint(0, len(nam_presets)-1)])
    nam.load_preset(preset_path)
    return nam

nam_t = get_nam()
nolly_t = get_nolly()
board = Pedalboard([nam_t, nolly_t])

#### Contruct list of data

In [12]:
goat_path = 'ENTER DATASET PATH'
csv_path = 'ENTER CSV PATH' ###
df = pd.read_csv(csv_path) ###

wav_files = []
for root, dirs, files in os.walk(goat_path):
    for f in files:
        if f.endswith('.wav') and not f.endswith('_gp.wav') and not f.endswith('_gp_di.wav'):
            path_audio = os.path.join(root, f)
            path_midi = os.path.join(root, f).replace('.wav', '_fine_aligned.mid')

            search_path = path_audio.replace(goat_path, '') ###
            matching_rows = df[df['audio_path'].str.contains(search_path, na=False, regex=False)] ###

            
            if len(matching_rows) != 1: ###
                continue ###

            f_measure_fine_value = matching_rows['f_measure_fine'].values[0]  ###
            if f_measure_fine_value < 0.75: ###
                continue ###
            
            # NOTE: The final published dataset may haev a different structure, so this will need to be updated
            # This is essentially just selected specific songs to be in the test split
            if 'Dani' in path_audio or 'Lithium' in path_audio or 'Reptilia' in path_audio:
                wav_files.append(('test', path_audio, path_midi))
            else:
                wav_files.append(('train', path_audio, path_midi))

#### Render data

In [13]:
goat_amp_path_test = 'ENTER YOUR DESIRED TEST PATH HERE'
goat_amp_path_train = 'ENTER YOUR DESIRED TRAIN PATH HERE'

i = 0
for f in tqdm(wav_files):
    split, path_audio, path_midi = f
    audio, sample_rate = librosa.load(path_audio, sr=None, mono=True)
    board = Pedalboard([get_nam(), get_nolly()])

    processed_audio = board(audio, sample_rate)
    processed_audio_name = hashlib.md5((path_audio + str(i)).encode()).hexdigest() + '.wav'
    i += 1
    if split == 'test':
        processed_audio_path = os.path.join(goat_amp_path_test, processed_audio_name)
        midi_path = processed_audio_path.replace('.wav', '.mid')
        try:
            os.system(f'cp "{path_midi}" "{midi_path}"')
        except:
            continue
        sf.write(processed_audio_path, processed_audio, sample_rate)
    elif split == 'train':
        processed_audio_path = os.path.join(goat_amp_path_train, processed_audio_name)
        midi_path = processed_audio_path.replace('.wav', '.mid')
        try:
            os.system(f'cp "{path_midi}" "{midi_path}"')
        except:
            continue
        sf.write(processed_audio_path, processed_audio, sample_rate)

100%|██████████| 222/222 [55:21<00:00, 14.96s/it] 
